In [36]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.feature_extraction import text
from sklearn.model_selection import GridSearchCV, KFold

1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.
5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [5]:
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'] )

In [41]:
X_raw = newsgroups['data']
y = newsgroups['target']

In [42]:
vectorizer = text.TfidfVectorizer()
X_transformed = vectorizer.fit_transform(X_raw)

In [43]:
feature_mapping = vectorizer.get_feature_names()
size = np.shape(feature_mapping)[0]
feature_mapping[:10]

['00',
 '000',
 '0000',
 '00000',
 '000000',
 '000021',
 '000050',
 '000062david42',
 '000406',
 '000410']

In [68]:
%%time
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X_transformed, y)

Wall time: 2min 16s


GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [84]:
score = 0
C = 0
results = pd.DataFrame(gs.cv_results_)
mean = results['mean_test_score']
parameters = results['params'][0]
for a in mean:
    if results['mean_test_score'][a] > score:
        score = results['mean_test_score'][a]
        C = results['params'][a]


TypeError: cannot do label indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [0.5526360264776301] of <class 'float'>

In [77]:
clf = SVC(kernel='linear', random_state=241, C=10.0)
clf.fit(X_transformed, y)

SVC(C=10.0, kernel='linear', random_state=241)

In [87]:
words = vectorizer.get_feature_names()
coef = pd.DataFrame(clf.coef_.data, clf.coef_.indices)
top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])
# top_words.sort()
def sortString(str) : 
    str = ','.join(sorted(str)) 
    print(str) 
sortString(top_words)


atheism,atheists,bible,god,keith,moon,nick,religion,sky,space
